# Modelling metabolism in soy mature leaves

### Generating SOY core model

In [1]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis

#import sbml file
fname = "Data/Soy_leaf_model_GA.sbml"
model = io.sbml.read_sbml_model(fname)



In [2]:
len(model.genes)

2098

### Importing transcript data

In [3]:
import pandas as pd
df = pd.read_csv("/home/sanu/Downloads/gmax_leaf_CO2_normExprn_1.txt",sep="\t")

/home/sanu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051 DtypeWarning: Columns (0,2,3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
amb_list = list(df.columns[0:17])
ele_list = list(df.columns[0:17])
for x in df.columns:
    if "Amb" in x:
        amb_list.append(x)
    elif "Ele" in x:
        ele_list.append(x)
    else:
        continue

print(amb_list)
print(ele_list)

['Glyma2.0', 'Query_Length', 'Uniport_ID', 'Hit_Alias', 'Hit_Length', 'E-Value', 'Alignment_Length', 'Query_Coverage', 'Percent_Identity', 'Identities_Covered_percent', 'Number_of_Matches', 'Bit_Score', 'HSPs', 'gene name/function', 'EC', 'ProbeID', 'Glyma 1.1', 'X474_E_Amb.CEL', 'X554_E_Amb.CEL', 'X560_E_Amb.CEL', 'X561_E_Amb.CEL', 'X580_E_Amb.CEL', 'X582_E_Amb.CEL', 'X583_E_Amb.CEL', 'X585_E_Amb.CEL']
['Glyma2.0', 'Query_Length', 'Uniport_ID', 'Hit_Alias', 'Hit_Length', 'E-Value', 'Alignment_Length', 'Query_Coverage', 'Percent_Identity', 'Identities_Covered_percent', 'Number_of_Matches', 'Bit_Score', 'HSPs', 'gene name/function', 'EC', 'ProbeID', 'Glyma 1.1', 'X460_E_Ele.CEL', 'X467_E_Ele.CEL', 'X468_E_Ele.CEL', 'X469_E_Ele.CEL', 'X556_E_Ele.CEL', 'X557_E_Ele.CEL', 'X558_E_Ele.CEL', 'X559_E_Ele.CEL']


In [5]:
df_amb = df[amb_list]
df_ele = df[ele_list]

In [6]:
modelGenes = dict()
for gene in model.genes:
    geneID = gene.id.split(".")[0]+"."+gene.id.split(".")[1]
    modelGenes[geneID]=gene


In [7]:
modelGenesNotInTranscript = set(modelGenes.keys()) - set(df["Glyma2.0"])

In [8]:
len(modelGenesNotInTranscript)

1094

In [9]:
fout = open("/home/sanu/SoyGenesInModel.csv","w")
for gene in model.genes:
    geneID = gene.id.split(".")[0]+"."+gene.id.split(".")[1]
    fout.write(geneID+"\n")
fout.close()

In [10]:
fout = open("/home/sanu/SoyGenesInModelMinusTranscriptomics.csv","w")
for gene in modelGenesNotInTranscript:
    geneID = gene.split(".")[0]+"."+gene.split(".")[1]
    fout.write(geneID+"\n")
fout.close()

In [11]:
i=0
for rxn in model.reactions:
    if len(rxn.genes)!=0:
        i=i+1
print(i)

802


In [13]:
leaf_model = model.copy()

for rxn in leaf_model.reactions:
    if rxn.lower_bound==-1000:
        rxn.lower_bound=-1500
    if rxn.upper_bound==1000:
        rxn.upper_bound=1500


PPFD = 1500
ATPase = (0.0049*PPFD) + 2.7851
leaf_model.reactions.get_by_id("ATPase_tx1").lower_bound = ATPase
leaf_model.reactions.get_by_id("ATPase_tx1").upper_bound = ATPase
leaf_model.reactions.diel_biomass.objective_coefficient=1
sol = flux_analysis.parsimonious.pfba(leaf_model)
print(leaf_model.reactions.CO2_tx1.flux)

from sweetlovegroup import analysis
temp = leaf_model.copy()
out_400 = analysis.estimateOutputFromNetCO2(temp,21.645569620253173)
leaf_model_400 = leaf_model.copy()
leaf_model_400.reactions.diel_biomass.lower_bound = out_400
leaf_model_400.reactions.diel_biomass.upper_bound = out_400
sol = flux_analysis.parsimonious.pfba(leaf_model_400)
print("Phloem output = ")
print(leaf_model_400.reactions.diel_biomass.flux)


temp = leaf_model.copy()
out_550 = analysis.estimateOutputFromNetCO2(temp,24.43037974683546)
leaf_model_550 = leaf_model.copy()
leaf_model_550.reactions.diel_biomass.lower_bound = out_550
leaf_model_550.reactions.diel_biomass.upper_bound = out_550
sol = flux_analysis.parsimonious.pfba(leaf_model_550)
print("Phloem output = ")
print(leaf_model_550.reactions.diel_biomass.flux)

34.66333506879305
500.0
10.1351
21.645569620253173
Phloem output = 
0.4524370423298771
500.0
10.1351
24.43037974683546
Phloem output = 
0.5210038706749772


In [18]:
for comp in leaf_model.compartments.keys():
    print(comp)
    print(leaf_model.compartments[comp])

m1

c1

b1

p1

v1

x1

r1

mi1

mc1

e1

l1

i1

c2

v2

e2

x2

b2

p2

m2

mi2

i2

mc2

r2

l2



In [18]:
ActiveRxnNotInTranscriptomicsData=set()
for rxn in leaf_model.reactions:
    if abs(rxn.flux) > 0 and len(rxn.genes)!=0:
        flag=False
        for gene in rxn.genes:
            if gene.id in set(df["Glyma2.0"]):
                flag=True
                print "Gene "+gene.id+"for rxn "+rxn.id+" present"
                break
        if not flag:
            ActiveRxnNotInTranscriptomicsData.add(rxn.id)

Gene Glyma.06G074900for rxn ACETYLGLUTKIN_RXN_p1 present
Gene Glyma.13G140700for rxn DIAMINOPIMDECARB_RXN_p1 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_p1 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_c1 present
Gene Glyma.20G079400for rxn NADH_DEHYDROG_A_RXN_mi1 present
Gene Glyma.13G097100for rxn RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1 present
Gene Glyma.14G053300for rxn IGPSYN_RXN_p1 present
Gene Glyma.10G016200for rxn FUMHYDR_RXN_m1 present
Gene Glyma.13G204700for rxn GLYCOGENSYN_RXN_p1 present
Gene Glyma.01G026700for rxn ALANINE_AMINOTRANSFERASE_RXN_c1 present
Gene Glyma.03G173400for rxn DIAMINOPIMEPIM_RXN_p1 present
Gene Glyma.10G064000for rxn THREDEHYD_RXN_p1 present
Gene Glyma.20G148100for rxn ACSERLY_RXN_c1 present
Gene Glyma.17G159700for rxn 2_ISOPROPYLMALATESYN_RXN_p1 present
Gene Glyma.14G217300for rxn 3_ISOPROPYLMALISOM_RXN_p1 present
Gene Glyma.12G042400for rxn GLUC1PADENYLTRANS_RXN_p1 present
Gene Glyma.20G148100for rxn ACSERLY_RXN_p1 present
Gene Glyma.1

Gene Glyma.09G167000for rxn SUCROSE_SYNTHASE_RXN_c1 present
Gene Glyma.10G211400for rxn CHORISMATE_SYNTHASE_RXN_p1 present
Gene Glyma.03G129700for rxn CYSTATHIONINE_BETA_LYASE_RXN_p1 present
Gene Glyma.13G153700for rxn PRIBFAICARPISOM_RXN_p1 present
Gene Glyma.03G212700for rxn METHYLENETHFDEHYDROG_NADP_RXN_c1 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_p2 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_c2 present
Gene Glyma.20G079400for rxn NADH_DEHYDROG_A_RXN_mi2 present
Gene Glyma.11G101400for rxn RXN_9663_p2 present
Gene Glyma.02G015700for rxn FUMHYDR_RXN_m2 present
Gene Glyma.02G015700for rxn FUMHYDR_RXN_c2 present
Gene Glyma.11G248700for rxn UREASE_RXN_c2 present
Gene Glyma.01G026700for rxn ALANINE_AMINOTRANSFERASE_RXN_c2 present
Gene Glyma.17G159700for rxn 2_ISOPROPYLMALATESYN_RXN_p2 present
Gene Glyma.14G217300for rxn 3_ISOPROPYLMALISOM_RXN_p2 present
Gene Glyma.20G148100for rxn ACSERLY_RXN_p2 present
Gene Glyma.11G101400for rxn RXN_9661_p2 present
Gene Glyma.12G1938

In [19]:
ActiveRxnNotInTranscriptomicsData

{'1_ACYLGLYCEROL_3_P_ACYLTRANSFER_RXN_p2',
 'GLUCOKIN_RXN_c2',
 'GLY3KIN_RXN_p1',
 'PHOSGLYPHOS_RXN_c1',
 'PHOSGLYPHOS_RXN_c2',
 'PHOSGLYPHOS_RXN_p1',
 'PHOSGLYPHOS_RXN_p2',
 'THREONINE_ALDOLASE_RXN_c1'}

In [20]:
df_amb[df_amb["Glyma2.0"]=="Glyma.14G053300"][df_amb.columns[17:]].mean().mean()

6.7630348276875

In [21]:
totalTranscript_400 = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_amb[df_amb["Glyma2.0"]==gene.id][df_amb.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_400[rxn.id]=x
    
totalTranscript_550 = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_ele[df_ele["Glyma2.0"]==gene.id][df_ele.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_550[rxn.id]=x

In [22]:
rxnList = list()
transcriptList = list()
fluxList = list()
import numpy as np

for rxnID in totalTranscript_550.keys():
    rxnList.append(rxnID)
    a=0
    b=0
    if round(totalTranscript_400[rxnID],3)==0 and round(totalTranscript_550[rxnID],3)==0:
        transcriptList.append(0)
    elif round(totalTranscript_400[rxnID],3)==0:
        transcriptList.append(np.nan)
    else:
        a = totalTranscript_550[rxnID]/totalTranscript_400[rxnID]
        transcriptList.append(a)
    if round(leaf_model_550.reactions.get_by_id(rxnID).flux,3)==0 and round(leaf_model_400.reactions.get_by_id(rxnID).flux,3)==0:
        fluxList.append(0)
    elif round(leaf_model_400.reactions.get_by_id(rxnID).flux,3)==0:
        fluxList.append(np.nan)
    else:
        b = leaf_model_550.reactions.get_by_id(rxnID).flux/leaf_model_400.reactions.get_by_id(rxnID).flux
        fluxList.append(b)
    if not (a==0 and b==0):
        if b == 0:
            continue
        if round(a/b,2)>0.98 and round(a/b,2)<1.02:
            print rxnID
            print a
            print b
            print a/b

df_ratios = pd.DataFrame(data={"rxn ID":rxnList,"Transcript ratio":transcriptList,"Flux ratio":fluxList})

INORGPYROPHOSPHAT_RXN_p1
1.101657328803641
1.08903172312
1.0115934232333557
NADH_DEHYDROG_A_RXN_mi2
1.0046849867090821
1.01675560787
0.9881282964439743
GAPOXNPHOSPHN_RXN_c2
1.0193884551113048
1.01191025142
1.007390184735083
PEPDEPHOS_RXN_c2
1.0215797817779837
1.01178551139
1.0096801844678556
FUMHYDR_RXN_m2
0.9982047427477271
1.00721935887
0.9910499971563462
TRIOSEPISOMERIZATION_RXN_p1
1.0072391861039773
1.01857028924
0.9888754823771729
RIB5PISOM_RXN_p2
0.979800200721292
0.994290325557
0.9854266661725176
PHOSPHOGLUCMUT_RXN_p2
1.0003436901517928
0.994290325557
1.0060881258110699
CITSYN_RXN_m2
1.0367447835558694
1.02202578301
1.0144017898517175
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi2
1.0057316875838567
1.00717613236
0.9985658468948113
TRANSALDOL_RXN_p2
1.0012607681881907
0.994223426223
1.0070782298827117
MALATE_DEH_RXN_c2
1.0152293808837947
1.01191025142
1.0032800630897292
MALATE_DEH_RXN_m2
1.0152293808837947
1.01737953002
0.9978865810950349
SUCCCOASYN_RXN_m2
1.010030296025145
1.007176

In [23]:
totalTranscript_400["INORGPYROPHOSPHAT_RXN_p1"]

111.53797876447916

In [ ]:
leaf_model.reactions.L_CITRULLINE_pc1.genes

In [ ]:
leaf_model_sink = model.copy()

leaf_model_sink = setupC3DielModel(leaf_model_sink,"MetabolitesToTransfer.txt",starch_sucrose_ratio=0.6)

met = Metabolite("X_Biomass_contribution_t1")
leaf_model_sink.reactions.Biomass_leaf_tx1.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_t1.remove_from_model()
leaf_model_sink.reactions.diel_biomass.add_metabolites({met:-3})

met = Metabolite("X_Biomass_contribution_t2")
leaf_model_sink.reactions.Biomass_leaf_tx2.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_t2.remove_from_model()
leaf_model_sink.reactions.diel_biomass.add_metabolites({met:-1})

leaf_model_sink.reactions.diel_biomass.objective_coefficient=1
flux_analysis.parsimonious.pfba(leaf_model_sink)
print("Biomass accumulation = ")
print(leaf_model_sink.reactions.diel_biomass.flux)

In [15]:
dir(rxn)

['__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_associate_gene',
 '_check_bounds',
 '_compartments',
 '_dissociate_gene',
 '_gene_reaction_rule',
 '_genes',
 '_id',
 '_lower_bound',
 '_metabolites',
 '_model',
 '_repr_html_',
 '_set_id_with_model',
 '_update_awareness',
 '_upper_bound',
 'add_metabolites',
 'annotation',
 'boundary',
 'bounds',
 'build_reaction_from_string',
 'build_reaction_string',
 'check_mass_balance',
 'compartments',
 'copy',
 'delete',
 'flux',
 'flux_expression',
 'forward_variable',
 'functio

In [31]:
met = leaf_model.metabolites.CARBON_DIOXIDE_p1
for rxn in met.reactions:
    sto = rxn.metabolites[met]
    if round(rxn.flux,3)!=0:
        print(rxn.reaction+"\t"+str(rxn.flux*sto))

CPD_659_p1 + PROTON_p1 --> CARBON_DIOXIDE_p1 + PHE_p1 + WATER_p1	0.01763617085978775
CARBON_DIOXIDE_p1 <=> CARBON_DIOXIDE_c1	41.45235659787356
AMMONIUM_p1 + 0.9 ATP_p1 + CARBON_DIOXIDE_p1 + 0.1 aATP_p1 <=> 0.8 ADP_p1 + CARBAMOYL_P_p1 + 1.9 PROTON_p1 + 0.2 aADP_p1	-0.006029628222136551
CARBON_DIOXIDE_p1 + D_RIBULOSE_15_P2_p1 + WATER_p1 + rubisco_bal_p1 --> 2.0 G3P_p1 + 2.0 PROTON_p1	-41.49445901899967
CPD_659_p1 + NADP_p1 --> CARBON_DIOXIDE_p1 + NADPH_p1 + TYR_p1	0.0009710390796269382
CARBOXYPHENYLAMINO_DEOXYRIBULOSE_P_p1 + PROTON_p1 --> CARBON_DIOXIDE_p1 + INDOLE_3_GLYCEROL_P_p1 + WATER_p1	0.0012947187728359173
PROTON_p1 + 2.0 PYRUVATE_p1 --> 2_ACETO_LACTATE_p1 + CARBON_DIOXIDE_p1	0.011206316736203542
2_OXOBUTANOATE_p1 + PROTON_p1 + PYRUVATE_p1 --> 2_ACETO_2_HYDROXY_BUTYRATE_p1 + CARBON_DIOXIDE_p1	0.00538883011378738
CPD_7100_p1 + PROTON_p1 --> 2K_4CH3_PENTANOATE_p1 + CARBON_DIOXIDE_p1	0.0047764628445533435
MESO_DIAMINOPIMELATE_p1 + PROTON_p1 --> CARBON_DIOXIDE_p1 + LYS_p1	0.0068585109